In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from keras.models import Sequential
from keras.layers import ConvLSTM2D, Conv3D, Dense, Flatten, GlobalAveragePooling2D, Conv2D, BatchNormalization, Reshape
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.backend import sum
from keras.layers import ConvLSTM2D, BatchNormalization, Conv3D, AveragePooling3D, Reshape,Dense,AveragePooling2D,GlobalAveragePooling2D, GlobalAveragePooling3D, Conv2D, MaxPooling2D, Flatten, TimeDistributed, Lambda, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import rmsprop, adam
from keras.models import load_model

Using TensorFlow backend.


In [ ]:
from keras import layers
from keras import Input
from keras.models import Model
from keras.utils import plot_model

In [4]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py

In [ ]:
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import gc
import tensorflow

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))

In [ ]:
h5f = h5py.File('./drive/My Drive/Colab Notebooks/ver2/next_frame/total_data_ro.h5','r')
train_data = h5f['total_2018'][:]
train_gen_data = h5f['gen_2018'][:]
val_data = h5f['total_2017'][:]
val_gen_data = h5f['gen_2017'][:]
h5f.close()

In [ ]:
train_data = np.rollaxis(train_data, 1, 4)
val_data = np.rollaxis(val_data, 1, 4)

In [9]:
sc = RobustScaler()
sc = joblib.load('./scaler_gen.pkl')
train_gen_data = sc.transform(train_gen_data.reshape(1, -1))
train_gen_data = train_gen_data.reshape(-1, 1)
val_gen_data = sc.transform(val_gen_data.reshape(1, -1))
val_gen_data = val_gen_data.reshape(-1, 1)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator RobustScaler from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
callbacks_list = [
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=10
    ),
    ModelCheckpoint(
        filepath='./gen_pred_120h_71h.h5',
        monitor='val_loss',
        save_best_only=True
    )
]

In [ ]:
def generator(data, target, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=1):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while True:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        
        samples = np.zeros((len(rows), lookback // step, data.shape[-3], data.shape[-2], data.shape[-1]))
        targets = np.zeros((len(rows), delay))
        
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = target[rows[j] - 1: rows[j] + delay - 1][:,0]
        #yield samples, targets
        yield samples, [targets[:,0], targets[:,1], targets[:,2], targets[:,3], targets[:,4], targets[:,5], targets[:,6], targets[:,7], targets[:,8], targets[:,9], targets[:,10], targets[:,11], targets[:,12], targets[:,13], targets[:,14], targets[:,15], targets[:,16], targets[:,17], targets[:,18], targets[:,19], targets[:,20], targets[:,21], targets[:,22], targets[:,23], targets[:,24], targets[:,25], targets[:,26], targets[:,27], targets[:,28], targets[:,29], targets[:,30], targets[:,31], targets[:,32], targets[:,33], targets[:,34], targets[:,35], targets[:,36], targets[:,37], targets[:,38], targets[:,39], targets[:,40], targets[:,41], targets[:,42], targets[:,43], targets[:,44], targets[:,45], targets[:,46], targets[:,47], targets[:,48], targets[:,49], targets[:,50], targets[:,51], targets[:,52], targets[:,53], targets[:,54], targets[:,55], targets[:,56], targets[:,57], targets[:,58], targets[:,59], targets[:,60], targets[:,61], targets[:,62], targets[:,63], targets[:,64], targets[:,65], targets[:,66], targets[:,67], targets[:,68], targets[:,69], targets[:,70], targets[:,71]]

In [ ]:
train_gen = generator(train_data, train_gen_data, 120, 72, 0, None, shuffle=True, batch_size=5)
val_gen = generator(val_data, val_gen_data, 120, 72, 0, None, shuffle=True, batch_size=5)

In [ ]:
reset_keras()

img_input = Input(shape=(None, 75, 75, 10), name='images')
convlstm2d = layers.ConvLSTM2D(filters=20, kernel_size=(3, 3),
                   input_shape=(None, 75, 75, 10),
                    data_format='channels_last', return_sequences=True,
                   padding='same')(img_input)
batch_norm = layers.BatchNormalization()(convlstm2d)

convlstm2d_1x1 = layers.ConvLSTM2D(filters=1, kernel_size=(3, 3),
                    data_format='channels_last',padding='same')(batch_norm)
batch_norm = layers.BatchNormalization()(convlstm2d_1x1)

flatten = layers.Flatten()(batch_norm)

#Dense_1_list = [layers.Dense(units=8, activation='relu')(flatten) for i in range(72)]

Dense_1_list = [layers.Dense(units=1)(flatten) for i in range(72)]

#Dense_2_list = [layers.Dense(units=1)(input_tensor) for input_tensor in Dense_1_list]

model = Model(img_input, Dense_1_list)

model.compile(optimizer=rmsprop(lr=0.001), loss=['mae' for i in range(72)])
model.summary()

In [ ]:
history = model.fit_generator(train_gen, validation_data=val_gen,
                              steps_per_epoch=25, validation_steps=25,
                             epochs=100, callbacks=callbacks_list)

In [ ]:
with open('./drive/My Drive/Colab Notebooks/ver2/next_frame/result.txt', 'wb') as f:
  pickle.dump(history.history, f)

In [ ]:
import pickle